<a href="https://colab.research.google.com/github/xpessoles/Informatique_Dev/blob/master/PyTorch_cifar10_tutorial_FormationCPGE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning pour la Classification d'Images

**Dans ce notebook, vous allez**:
- Apprendre les bases de PyTorch, un progiciel puissant mais facile à utiliser pour le calcul scientifique (et le deep learning)
- Apprendre à créer et à paramétrer un réseau de neurones de convolution pour la classification des images.

## Preliminaires

Le commandes suivantes définissent des fonctions d'affichage et importent les fonctions nécessaires au tutoriel. Exécutez simplement la cellule avant de débuter le tutoriel.

In [ ]:
%matplotlib inline

# Python 2/3 compatibility
from __future__ import print_function, division

import itertools
import time


import numpy as np
import matplotlib.pyplot as plt

# Colors from Colorbrewer Paired_12
colors = [[31, 120, 180], [51, 160, 44]]
colors = [(r / 255, g / 255, b / 255) for (r, g, b) in colors]

# functions to show an image
def imshow(img):
    """
    :param img: (PyTorch Tensor)
    """
    # unnormalize
    img = img / 2 + 0.5     
    # Convert tensor to numpy array
    npimg = img.numpy()
    # Color channel first -> color channel last
    plt.imshow(np.transpose(npimg, (1, 2, 0)))



def plot_losses(train_history, val_history):
    x = np.arange(1, len(train_history) + 1)

    plt.figure(figsize=(8, 6))
    plt.plot(x, train_history, color=colors[0], label="Training loss", linewidth=2)
    plt.plot(x, val_history, color=colors[1], label="Validation loss", linewidth=2)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.title("Evolution of the training and validation loss")
    plt.show()

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    :param cm: (numpy matrix) confusion matrix
    :param classes: [str]
    :param normalize: (bool)
    :param title: (str)
    :param cmap: (matplotlib color map)
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
    plt.figure(figsize=(8, 8))   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# I. Qu'est-ce que PyTorch ?

C'est une bibliothèque de calcul scientifique en Python avec deux objectif principaux:

- Replacer Numpy (la bibliothèque de calcul numérique de python) pour permettre l'exploitation des GPUs (processeurs de cartes graphiques)
- Fournir une plateforme de recheche pour le deep learning efficace et simple à utiliser



## 1. Tenseurs PyTorch 

Un tenseur (`torch.Tensor`) est une matrice multi-dimensionelle contenant des données de même type.

Les `Tensor` sont similaires aux `ndarrays` de numpy, mais ils peuvent être utilisés pour du calcul sur GPU.

### Tensor Shape

Pour connaître la forme d'un tenseur donné, vous pouvez utiliser la méthode `.size ()` (l'équivalent numpy est `.shape`)

In [ ]:
import numpy as np
# Import torch and create the alias "th"
# instead of writing torch.name_of_a_method() , we only need to write th.name_of_a_method()
# (similarly to numpy imported as np)
import torch as th

# Create tensor of ones (FloatTensor by default)
ones = th.ones(3, 2)
print(ones)


# Display the shape of a tensor
# it can be used as a tuple
print("Tensor Shape: {}".format(ones.size()))

tensor([[1., 1.],
        [1., 1.],
        [1., 1.]])
Tensor Shape: torch.Size([3, 2])


### Reshape tensors

Pour remodeler les tenseurs (par exemple, aplatir un tenseur 3D en un tableau 1D), vous pouvez utiliser la méthode `.view ()`:

- **x.view(new_shape)**: renvoie un nouveau tenseur avec les mêmes données mais une taille différente. C'est l'équivalent de la fonction numpy * reshape * (Donne une nouvelle forme à un tableau sans changer ses données.). Vous pouvez lire la documentation complète [ici.](Http://pytorch.org/docs/master/tensors.html#torch.Tensor.view)

[ATTENTION] lorsque vous spécifiez une nouvelle forme, vous devez vous assurer que le nombre d'éléments est constant. Par exemple, une matrice 2D de taille 3x3 ne peut être vue que comme un tableau 1D de taille $ 3 x 3 = 9 $

In [ ]:
# Create a 3D tensor of size 3x2x2
zeros_3d_tensor = th.zeros(3, 2, 2)
print("Original size:", zeros_3d_tensor.size())

# Reshape it to a 1D array of size 3*2*2 = 12
zeros_1d_array = zeros_3d_tensor.view(3 * 2 * 2)
print("Reshaped tensor:", zeros_1d_array.size())


# Let's view our original tensor as a 2D matrix
# If you want PyTorch to guess one remaining dimension,
# you specify '-1' instead of the actual size
zeros_2d_matrix = zeros_3d_tensor.view(-1, 2 * 2)

print("Matrix shape:", zeros_2d_matrix.size())

Original size: torch.Size([3, 2, 2])
Reshaped tensor: torch.Size([12])
Matrix shape: torch.Size([3, 4])


### Opérations de base sur les tensors

Le tenseur prend en charge toutes les opérations de base d’algèbre linéaire. Vous pouvez lire la documentation complète [ici](http://pytorch.org/docs/master/tensors.html)

In [ ]:
2 * ones + 1

tensor([[3., 3.],
        [3., 3.],
        [3., 3.]])

Les tenseurs PyTorch supportent également l'indexation numpy:

In [ ]:
print("\n Indexing Demo:")
print(ones[:, 1])


 Indexing Demo:
tensor([1., 1., 1.])


## 2. Pont avec Numpy
AVERTISSEMENT: les tenseurs PyTorch sont différents des tableaux numpy
même s'ils ont beaucoup en commun

Cependant, il est facile avec PyTorch de transformer des tableaux Tenseurs en tableaux Numpy et vice versa:

### Numpy <-> PyTorch

La création des tenseurs PyTorch à partir du tableau numpy se fait via la fonction `torch.from_numpy ()` (ici `th.from_numpy ()` car nous avons renommé *torch* en *th*)

Pour transformer un tenseur PyTorch en un tableau numpy, vous pouvez simplement appeler la méthode `.numpy ()`.

In [ ]:
# np.float32 -> th.FloatTensor
ones_matrix = np.ones((2, 2), dtype=np.float32)

# the matrix is passed by reference:
# if we modify the original numpy array, the tensor is also edited
ones_tensor = th.from_numpy(ones_matrix)
# Convert back to a numpy matrix
numpy_matrix = ones_tensor.numpy()

print("PyTorch Tensor:")
print(ones_tensor)

print("Numpy Matrix:")
print(numpy_matrix)

PyTorch Tensor:
tensor([[1., 1.],
        [1., 1.]])
Numpy Matrix:
[[1. 1.]
 [1. 1.]]


## 3. Différentiation Automatique

Les tenseurs de Pytorch permettent de **calculer automatiquement les gradients**. Cela est particulièrement utile pour la rétropropagation.

Une fois que vous avez terminé votre calcul, vous pouvez appeler `.backward()` et faire en sorte que tous les gradients soient calculés automatiquement.

Vous pouvez accéder au gradient par rapport à une variable en utilisant `.grad`.

Pour illustrer l’utilisation des `Variable` PyTorch,
définissons une simple transformation linéaire d'une variable $x$:
$$y = a \cdot x + b$$

PyTorch nous permet de calculer automatiquement $$ \frac{dy}{dx} $$

In [ ]:
# Create a tensor and tell PyTorch
# that we want to compute the gradient
# We need to specify that we want to compute the gradient
# as it requires extra memory and computation

x = th.ones(1, requires_grad=True)

# Transformation constants
a = 2
b = 5

# Define the tranformation and store the result
# in a new variable
y = a * x + b

print(y)

tensor([7.], grad_fn=<AddBackward0>)


Calculons les gradients :

In [ ]:
y.backward()

`x.grad` contient le gradient:

$$\frac{dy}{dx} = a$$

puisque $y = a \cdot x + b$

In [ ]:
x.grad

tensor([2.])

Vous pouvez maintenant changer les valeurs de $ a $ et $ b $ pour voir leurs effets sur le gradient (et constater que `x.grad` ne dépend que de la valeur de `a`)

## 4. PyTorch et les GPU (support CUDA)

Google colab fournit un processeur graphique compatible CUDA, nous allons donc utiliser ses capacités. Vous pouvez déplacer le tenseur vers le GPU en utilisant simplement la méthode `to()`. Sinon, PyTorch utilisera le processeur.

Ici, nous allons démontrer l’utilité du GPU sur une simple multiplication de matrice:

In [ ]:
import time

if th.cuda.is_available():
  # Create tensors
  x = th.ones(1000, 1000)
  y = 2 * x + 3
  # Do the calculation on cpu (default)
  start_time = time.time()
  # Matrix multiplication (for benchmark purpose)
  results = th.mm(x, y)
  time_cpu = time.time() - start_time
  
  # Do the same calculation but on the gpu
  # First move tensors to gpu
  x = x.to("cuda")
  y = y.to("cuda")
  start_time = time.time()
  # Matrix multiplication (for benchmark purpose)
  results = th.mm(x, y)
  time_gpu = time.time() - start_time
  
  print("Time on CPU: {:.5f}s \t Time on GPU: {:.5f}s".format(time_cpu, time_gpu))
  print("Speed up: Computation was {:.0f}X faster on GPU!".format(time_cpu / time_gpu))
  
else:
  print("You need to enable GPU accelaration in colab (runtime->change runtime type)")

Time on CPU: 0.07276s 	 Time on GPU: 0.00420s
Speed up: Computation was 17X faster on GPU!


Comme prévu, la multiplication de matrice est beaucoup plus rapide sur un GPU, il est donc préférable de l'utiliser.


# II. Entrainement d'un classifieur


Pour ce tutoriel, nous allons utiliser le jeu de données CIFAR10.
Il existe 10 classes: «avion», «automobile», «oiseau», «chat», «cerf»,
«Chien», «grenouille», «cheval», «navire», «camion». Les images dans CIFAR-10 sont de taille 3x32x32, c’est-à-dire des images couleur à 3 canaux de 32x32 pixels.

![CIFAR10](http://pytorch.org/tutorials/_images/cifar10.png)

Nous allons faire les étapes suivantes dans l'ordre:

1. Charger et normaliser les jeux de données d'entrainement et de test CIFAR10 à l’aide de `` torchvision``
2. Définir un réseau de neurones de convolution
3. Définir une fonction de perte
4. Entrainer le réseau sur les données de d'entrainement
5. Testez le réseau sur les données de test

## 1. Charger and normaliser le dataset CIFAR10 

Avec `` torchvision``, il est relativement facile de charger CIFAR10 a l'aide des fonctions fournies :
* la fonction 'datasets.CIFAR10' chargent directement les données depuis un dépot public. Elles peuvent leur appliquer des transformations, ici une normalisation sur la place [-1,1].
* les fonctions `sampler` permettent de prendre une partie des données (ici aléatoirement)
* les `DataLoader` sont ensuite définis. Il permettront à l'algorithme d'apprentissage ou de test d'obtenir des données au format adapté (bonne taille de batch par exemple)

Commençons par définir les `sampler` :

In [ ]:
import torch
import torchvision

# Define default device, we should use the GPU (cuda) if available
device = th.device("cuda" if th.cuda.is_available() else "cpu")

from torch.utils.data.sampler import SubsetRandomSampler

n_training_samples = 20000 # Max: 50 000 - n_val_samples
n_val_samples = 5000
n_test_samples = 5000

train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

Ensuite, définissons les transformations à appliquer lors du chargement :



In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

Créons ensuite l'interface avec le dataset et les `Dataloader`:


In [ ]:
num_workers = 2
test_batch_size = 4

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=test_batch_size, sampler=train_sampler,
                                          num_workers=num_workers)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=test_batch_size, sampler=test_sampler,
                                         num_workers=num_workers)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Comme premier exemple d'utilisation des `Dataloader`, nous pouvons afficher des images aléatoires du dataset:

In [ ]:
# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('{:>10}'.format(classes[labels[j]]) for j in range(test_batch_size)))

## 2. Definition d'un réseau de neurones convolutionnel

Commençons par importer les modules utiles :

In [ ]:
# Useful imports
import torch.nn as nn
import torch.nn.functional as F


En PyTorch, des fonctions intégrées permettent de définir les différents éléments d'un réseau de convolution:

- **nn.Conv2d(in_channels, out_channels, kernel_size, stride = 1, padding = 0):** Couche de convolution. Vous pouvez lire la documentation complète [ici](http://pytorch.org/docs/master/nn.html#conv2d)

- **nn.MaxPool2d(taille du noyau, stride = aucun, remplissage = 0):** Couche de max-pooling. Vous pouvez lire la documentation complète [ici](http://pytorch.org/docs/master/nn.html#maxpool2d)

- **F.relu(Z1):** applique la fonction ReLU pour chaque élément de Z1 (qui peut être n’importe quelle forme). Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/nn.html#torch.nn.ReLU)

- **x.view(new_shape)**: renvoie un nouveau tenseur avec les mêmes données mais une taille différente. C'est l'équivalent de la fonction numpy *reshape* (donne une nouvelle forme à un tableau sans changer ses données). Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/tensors.html#torch.Tensor.view)

- **nn.Linear(in_features, out_features)**: Applique une transformation linéaire aux données entrantes: $y = Ax + b$, elle est également appelée  'couche entièrement connectée'. Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/nn.html#linear-layers)

Nous allons maintenant définir un réseau simples, avec une chouche de convolution suivie d'un max pooling, une couche entièrement connectée et une couche de sortie:

In [ ]:
class SimpleConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(SimpleConvolutionalNetwork, self).__init__()
        
        # Initialize the network layers in the right order

        self.conv1 = nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # cf comments in forward() to have step by step comments
        # on the shape (how we pass from a 3x32x32 input image to a 18x16x16 volume)
        self.fc1 = nn.Linear(18 * 16 * 16, 64) 
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        """
        Forward pass,
        x shape is (batch_size, 3, 32, 32)
        (color channel first)
        in the comments, we omit the batch_size in the shape
        """
        # Here we define the operations that are required to compute the network output from the input data

        # shape : 3x32x32 -> 18x32x32
        x = F.relu(self.conv1(x))
        # 18x32x32 -> 18x16x16
        x = self.pool(x)
        # 18x16x16 -> 4608
        x = x.view(-1, 18 * 16 * 16)
        # 4608 -> 64
        x = F.relu(self.fc1(x))
        # 64 -> 10
        # The softmax non-linearity is applied later (cf createLossAndOptimizer() fn)
        x = self.fc2(x)
        return x

## 3. Définition d'une fonction de cout et d'un optimiseur

Nous utilisons une fonction de cout à base de cross-entropie pour la classification et ADAM. Vous pouvez en savoir plus sur les [méthodes d'optimisation](https://pytorch.org/docs/stable/optim.html).


In [ ]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    # it combines softmax with negative log likelihood loss
    criterion = nn.CrossEntropyLoss()  
    #optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    return criterion, optimizer

## 4. Entrainement du réseau

Nous allons maintenant utiliser tous ces éléments pour réaliser l'entrainement. Pour cela, nous créons une fonction qui réalise une boucle qui utilise le `DataLoader`, applique le réseau sur les données obtenues, and optimise les poids afin de réduire la `loss` calculée.



### Définition de la fonction d'entrainement


In [ ]:
def get_train_loader(batch_size):
    return torch.utils.data.DataLoader(train_set, batch_size=batch_size, sampler=train_sampler,
                                              num_workers=num_workers)

# Use larger batch size for validation to speed up computation
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler,
                                          num_workers=num_workers)


def train(net, batch_size, n_epochs, learning_rate):
    """
    Train a neural network and print statistics of the training
    
    :param net: (PyTorch Neural Network)
    :param batch_size: (int)
    :param n_epochs: (int)  Number of iterations on the training set
    :param learning_rate: (float) learning rate used by the optimizer
    """
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("n_epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    train_loader = get_train_loader(batch_size)
    n_minibatches = len(train_loader)

    criterion, optimizer = createLossAndOptimizer(net, learning_rate)
    # Init variables used for plotting the loss
    train_history = []
    val_history = []

    training_start_time = time.time()
    best_error = np.inf
    best_model_path = "best_model.pth"
    
    # Move model to gpu if possible
    net = net.to(device)

    for epoch in range(n_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        print_every = n_minibatches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, (inputs, labels) in enumerate(train_loader):

            # Move tensors to correct device
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            total_train_loss += loss.item()

            # print every 10th of epoch
            if (i + 1) % (print_every + 1) == 0:    
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                      epoch + 1, int(100 * (i + 1) / n_minibatches), running_loss / print_every,
                      time.time() - start_time))
                running_loss = 0.0
                start_time = time.time()

        train_history.append(total_train_loss / len(train_loader))

        total_val_loss = 0
        # Do a pass on the validation set
        # We don't need to compute gradient,
        # we save memory and computation using th.no_grad()
        with th.no_grad():
          for inputs, labels in val_loader:
              # Move tensors to correct device
              inputs, labels = inputs.to(device), labels.to(device)
              # Forward pass
              predictions = net(inputs)
              val_loss = criterion(predictions, labels)
              total_val_loss += val_loss.item()
            
        val_history.append(total_val_loss / len(val_loader))
        # Save model that performs best on validation set
        if total_val_loss < best_error:
            best_error = total_val_loss
            th.save(net.state_dict(), best_model_path)

        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))

    print("Training Finished, took {:.2f}s".format(time.time() - training_start_time))
    
    # Load best model
    net.load_state_dict(th.load(best_model_path))
    
    return train_history, val_history

### Entrainement

Nous allons maintenant pouvoir lancer l'entrainement. Cela devrait prendre une dizaine de secondes par époque :

In [ ]:
net = SimpleConvolutionalNetwork()

train_history, val_history = train(net, batch_size=32, n_epochs=3, learning_rate=0.001)

### Visualisation de l'évolution de la `loss`

Nous pouvons mainentant visualiser l'évolution des pertes d'entrainement et de validation

In [ ]:
plot_losses(train_history, val_history)

## 5. Test du réseau sur les données de test


Nous avons entrainé le réseau pour 3 passages sur l'ensemble de données d'apprentissage. Mais nous devons vérifier si le réseau a appris quelque chose.

Nous allons vérifier cela en prédisant l'étiquette de classe donnée par le réseau de neurones, et la comparer à la vérité sur le terrain. Si la prédiction est correct, nous ajoutons l'échantillon à la liste des prédictions correctes.

Commençons par charger et afficher les données de test:

In [ ]:
try:
  images, labels = next(iter(test_loader))
except EOFError:
  pass

# print images
imshow(torchvision.utils.make_grid(images))
print("Ground truth:\n")

print(' '.join('{:>10}'.format(classes[labels[j]]) for j in range(test_batch_size)))

### Résultats sur les données de test

Voyons maintenant les prédiction du réseau pour ces images. Les sorties correspondent aux probabilités de chacune des 10 classes. Il faut donc chercher la classe de plus forte probabilité:



In [ ]:
outputs = net(images.to(device))
print(outputs.size())

_, predicted = torch.max(outputs, 1)

print("Predicted:\n")
imshow(torchvision.utils.make_grid(images))

print(' '.join('{:>10}'.format(classes[predicted[j]]) for j in range(test_batch_size)))

### Performances en précision

Voyons comment le réseau fonctionne sur l’ensemble des tests.


In [ ]:
def dataset_accuracy(net, data_loader, name=""):
    net = net.to(device)
    correct = 0
    total = 0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
    accuracy = 100 * float(correct) / total
    print('Accuracy of the network on the {} {} images: {:.2f} %'.format(total, name, accuracy))

def train_set_accuracy(net):
    dataset_accuracy(net, train_loader, "train")

def val_set_accuracy(net):
    dataset_accuracy(net, val_loader, "validation")  
    
def test_set_accuracy(net):
    dataset_accuracy(net, test_loader, "test")

def compute_accuracy(net):
    train_set_accuracy(net)
    val_set_accuracy(net)
    test_set_accuracy(net)
    
print("Computing accuracy...")
compute_accuracy(net)

Les performances sur l’ensemble de test des images sont nettement meilleures que le hasard, qui correspond à une précision de 10% (sélection aléatoire d'une classe sur 10 classes). Pour information, un modèle linéaire atteint une précision d'environ 30%.

Nous pouvons ensuite regarder quelles sont les classes qui ont été bien reconnues ou non, et calculer les données de la matrice de confusion au passage :


In [ ]:
def accuracy_per_class(net):
    net = net.to(device)
    n_classes = 10
    # (real, predicted)
    confusion_matrix = np.zeros((n_classes, n_classes), dtype=np.int64)

    for images, labels in test_loader:
        images, labels = images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for i in range(test_batch_size):
            confusion_matrix[labels[i], predicted[i]] += 1
            label = labels[i]

    print("{:<10} {:^10}".format("Class", "Accuracy (%)"))
    for i in range(n_classes):
        class_total = confusion_matrix[i, :].sum()
        class_correct = confusion_matrix[i, i]
        percentage_correct = 100.0 * float(class_correct) / class_total
        
        print('{:<10} {:^10.2f}'.format(classes[i], percentage_correct))
    return confusion_matrix

confusion_matrix = accuracy_per_class(net)

### Visualisation de la matrice de Confusion

Regardons quel type d'erreur faite notre réseau.
Il semble que notre réseau est assez bon pour classer les navires,
mais a quelques difficultés à différencier les chats et les chiens.
En outre, beaucoup de camions sont classés comme des voitures.

In [ ]:
# Plot normalized confusion matrix
plot_confusion_matrix(confusion_matrix, classes, normalize=True,
                      title='Normalized confusion matrix')

# Plot non-normalized confusion matrix
plot_confusion_matrix(confusion_matrix, classes,
                      title='Confusion matrix, without normalization')

# III. Exploration des architectures de CNN

Maintenant, c'est à votre tour de construire un réseau de neurones convolutionnels. L'objectif de cette section est d'explorer différentes architectures CNN et de définir des hyperparamètres afin d'obtenir la meilleure précision possible sur l'ensemble de **test** !

Le réseau que vous devez modifier s'appelle **MyConvolutionalNetwork**.

Vous pouvez commencer à modifier la valeur batch_size, le nombre d'époques, la taille du test et d el'entrainement, puis essayer d'ajouter d'autres couches de convolution.

## Rappel des fonctions PyTorch utiles

Nous rappelons ici les fonctions utiles pour définir un réseau :

- **nn.Conv2d(in_channels, out_channels, kernel_size, stride = 1, padding = 0):** Couche de convolution. Vous pouvez lire la documentation complète [ici](http://pytorch.org/docs/master/nn.html#conv2d)

- **nn.MaxPool2d(taille du noyau, stride = aucun, remplissage = 0):** Couche de max-pooling. Vous pouvez lire la documentation complète [ici](http://pytorch.org/docs/master/nn.html#maxpool2d)

- **F.relu(Z1):** applique la fonction ReLU pour chaque élément de Z1 (qui peut être n’importe quelle forme). Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/nn.html#torch.nn.ReLU)

- **x.view(new_shape)**: renvoie un nouveau tenseur avec les mêmes données mais une taille différente. C'est l'équivalent de la fonction numpy *reshape* (donne une nouvelle forme à un tableau sans changer ses données). Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/tensors.html#torch.Tensor.view)

- **nn.Linear(in_features, out_features)**: Applique une transformation linéaire aux données entrantes: $y = Ax + b$, elle est également appelée  'couche entièrement connectée'. Vous pouvez lire la documentation complète [ici.](http://pytorch.org/docs/master/nn.html#linear-layers)

### Formules de convolution

Pour vous aider, voici les formules reliant la taille de sortie $(C_2, H_2, W_2)$ d'une couche de convolution dont l'entrée est de taille $(C_1, H_1, W_1)$ :


$$ H_2 = \lfloor \frac{H_1 - kernel\_size + 2 \times padding}{stride} \rfloor +1 $$

$$ W_2 = \lfloor \frac{W_1 - kernel\_size + 2 \times padding}{stride} \rfloor +1 $$

$$ C_2 = \text{number of filters used in the convolution}$$

NOTE: $C_2 = C_1$ pour le max pooling

avec:
- $H_2$: hauteur  
- $W_2$: largeur
- $C_1$: nombre de canaux d'entrée
- $C_2$: nombre de canaux de sortie

In [ ]:
def get_output_size(in_size, kernel_size, stride=1, padding=0):
    """
    Get the output size given all the parameters of the convolution
    :param in_size: (int) input size
    :param kernel_size: (int)
    :param stride: (int)
    :param paddind: (int)
    :return: (int)
    """
    return int((in_size - kernel_size + 2 * padding) / stride) + 1

Exemple d'utilisation de get_output_size() : Supposons que vous avez un *tenseur d'entrée de taille 3x32x32* (où 3 est le nombre de canaux) et vous utilisez une convolution 2D avec les paramètres suivants:
```python
conv1 = nn.Conv2d(3, 18, kernel_size=7, stride=2, padding=1)
```
alors, la taille du volume de sortie est 18 x ? x ? (parce que nous avons 18 filtres) où ? est donnée par les formules de convolution (voir ci-dessus).
**get_output_size()** permet de calculer cette valeur :

```
out_size = get_output_size(in_size=32, kernel_size=7, stride=2, padding=1)
print(out_size) # prints 14
```

Le couche de sortie est donc de taille 18x14x14

In [ ]:
out_size = get_output_size(in_size=32, kernel_size=7, stride=2, padding=1)
print(out_size)

## Définition et entrainement de votre réseau



### Définiton du réseau

Voici le réseau que vous pouvez modifier. Il est pour le moment constitué d'une seule couche de convolution, d'une couche de pooling et d'un couche entièrement connectée :

In [ ]:
class MyConvolutionalNetwork(nn.Module):
    def __init__(self):
        super(MyConvolutionalNetwork, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        #### START CODE: ADD NEW LAYERS ####
        # (do not forget to update `flattened_size`:
        # the input size of the first fully connected layer self.fc1)
        # self.conv2 = ...
        
        # Size of the output of the last convolution:
        self.flattened_size = 18 * 16 * 16
        ### END CODE ###
        
        self.fc1 = nn.Linear(self.flattened_size, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        """
        Forward pass,
        x shape is (batch_size, 3, 32, 32)
        (color channel first)
        in the comments, we omit the batch_size in the shape
        """
        # shape : 3x32x32 -> 18x32x32
        x = F.relu(self.conv1(x))
        # 18x32x32 -> 18x16x16
        x = self.pool(x)
        
        #### START CODE: USE YOUR NEW LAYERS HERE ####
        # x = ...
        
        #### END CODE ####
        
        # Check the output size
        output_size = np.prod(x.size()[1:])
        assert output_size == self.flattened_size,\
                "self.flattened_size is invalid {} != {}".format(output_size, self.flattened_size)
        
        # 18x16x16 -> 4608
        x = x.view(-1, self.flattened_size)
        # 4608 -> 64
        x = F.relu(self.fc1(x))
        # 64 -> 10
        x = self.fc2(x)
        return x

### Entrainement

Vous pouvez ensuite l'entrainer :

In [ ]:
net = MyConvolutionalNetwork()
train_history, val_history = train(net, batch_size=32, n_epochs=10, learning_rate=0.001)

### Affichage de la `Loss`

In [ ]:
plot_losses(train_history, val_history)

### Précision du modèle

Pour mémoire, le réseau défini dans la partie II avait une performance autour de 60%

In [ ]:
compute_accuracy(net)

In [ ]:
confusion_matrix = accuracy_per_class(net)

In [ ]:
plot_confusion_matrix(confusion_matrix, classes,
                      title='Confusion matrix, without normalization')

# Pour aller plus loin

- [Coursera Course on CNN](https://www.coursera.org/learn/convolutional-neural-networks)
- [Standford Course](http://cs231n.stanford.edu/syllabus.html)
- [PyTorch Tutorial](http://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)



Remarque: Ce tutoriel est basé sur le [tutoriel original de PyTorch](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) et a été adapté par [Antonin Raffin](http://araffin.github.io) et David Filliat. 